In [87]:
# -*- coding: UTF-8 -*-
import requests
import random
import json
import time
import csv
import os
import MySQLdb
from bs4 import BeautifulSoup
# 主地址
url = 'https://zhaopin.baidu.com/api/qzasync'
# 代理池
agent = [
        'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0',
        'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
        'Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
        'Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11',
        'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
]
# 代理ip池
proxies_agent = []

# 请求字段
params = {
    'query': '大数据',
    'city': '西安',
    'is_adq':'1',
    'pcmod':'1',
    'token':'==QxQvtqR6tqYhFcrhplsxmZud5kIi1au15bvt2kUuJl',
    'pn':320,
    'rn':20
}

# 插入sql
insert_sql = """INSERT INTO job_list (
                title,
                jobfirstclass,
                second_level_label,
                experience,
                jobsecondclass,
                enddate,
                size,
                education,
                first_level_label,
                officialname,
                type,
                salary,
                requirements,
                companydescription,
                loc
            ) VALUES (
                '{title}',
                '{jobfirstclass}',
                '{second_level_label}',
                '{experience}',
                '{jobsecondclass}',
                '{enddate}',
                '{size}',
                '{education}',
                '{first_level_label}',
                '{officialname}',
                '{type}',
                '{salary}',
                '{requirements}',
                '{companydescription}','{loc}'
            )
            """
# csv格式头部
csv_headers = [
    'title', 
    'jobfirstclass',
    'second_level_label',
    'experience',
    'jobsecondclass',
    'enddate',
    'size',
    'education',
    'first_level_label',
    'officialname',
    'type',
    'salary',
    'requirements',
    'companydescription',
    'loc',
]

In [101]:
class Utils(object):
    """
    工具类
    """
    def get_headers(self, new_headers={}):
        """
        生成请求头
        :new_headers:更新请求头部分
        """
        headers = {
            'User-Agent': random.sample(agent, 1)[0],
            'Cookie':'PSTM=1566896187; BIDUPSID=E446B6B0B9A3CCD4C0CDA8E1893F9A11; BAIDUID=85AD01D13D4709BCD0A64450272180BF:FG=1; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; BDRCVFR[feWj1Vr5u3D]=I67x6TjHwwYf0; delPer=0; PSINO=7; BCLID=6806718698358509382; BDSFRCVID=JokOJeC62mvfILnuI5V9UGfT_F_TvecTH6aolZwWe98cc5_BsOm2EG0PoU8g0KubAD-9ogKK3gOTH48F_2uxOjjg8UtVJeC6EG0Ptf8g0f5; H_BDCLCKID_SF=tb4D_CIyfII3j5rFMtJh-tCsbMQ02PFXKKOLVhcO2h7keq8CD4c-bxFThloAtlK8Qmvh-U7IXMb-eJO2y5jHhPAl5nJ-q6v2KbTd5tOvaCopsIJMhp_WbT8U5f5C0Iu8aKviaKOjBMb1MKnDBT5h2M4qMxtOLR3pWDTm_q5TtUJMeCnTD-Dhe4tX-NFetjk8JMK; H_PS_PSSID=31728_1440_31670_21115_31781_31673_32025_31715_30824_31845; ZP_FLAG=12; BDUSS=kl-a3NSclhqd1pkfkVTYWtxZGdLUFdHbkN4Y3VwcTZ6NFA4bUpwSWhFWkFId2hmSUFBQUFBJCQAAAAAAAAAAAEAAABglYasdrn-uf65~r7kdXUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAECS4F5AkuBec'
        }
        headers.update(new_headers)
        return headers
    def get_path(self):
        """
        获取本地路径
        """
        return os.getcwd()
    def get_insert_sql(self, txt):
        """
        生成插入sql语句
        :txt:数据
        """
        sql = insert_sql.format(title = txt.get('title') ,jobfirstclass = txt.get('jobfirstclass'),second_level_label = txt.get('second_level_label'),
                experience = txt.get('experience'),jobsecondclass = txt.get('jobsecondclass'),enddate = txt.get('enddate'),
                size = txt.get('size'),education = txt.get('education'),first_level_label = txt.get('first_level_label'),
                officialname = txt.get('officialname'),type = txt.get('type'),salary = txt.get('salary'),
                requirements = txt.get('requirements').split(':')[1],companydescription = txt.get('companydescription'),loc = txt.get('loc'))
        return sql
    def parse_dict(self, txt):
        """
        筛选字段
        :txt:数据
        """
        data = {
            'title': txt.get('title'),
            'jobfirstclass':txt.get('jobfirstclass'),
            'second_level_label':txt.get('second_level_label'),
            'experience':txt.get('experience'),
            'jobsecondclass':txt.get('jobsecondclass'),
            'enddate':txt.get('enddate'),
            'size':txt.get('size'),
            'education':txt.get('education'),
            'first_level_label':txt.get('first_level_label'),
            'officialname':txt.get('officialname'),
            'type':txt.get('type'),
            'salary':txt.get('salary'),
            'requirements':txt.get('requirements').split(':')[1],
            'companydescription':txt.get('companydescription'),
            'loc':txt.get('loc')
        }
        return data
    def get_page_url(self,loc):
        """
        生成详细页请求信息
        :loc:参数id
        """
        data = {
            'url':'https://zhaopin.baidu.com/szzw',
            'params': {
                'id': loc,
                'query':'大数据',
                'city':'西安',
                'is_promise':1,
                'vip_sign':'',
                'asp_ad_job':''
            }
        }
        return data
    def get_proxies(self):
        """
        读取csv文件获取代理ip
        """
        with open('ip.csv', 'r') as f:
            reader = csv.reader(f)
            for row in reader:
                proxies_agent.append(row)
        print("获取代理ip成功！！！")
    def get_proxies_agent(self):
        """
        生成代理ip头部
        """
        tmp = random.sample(proxies_agent, 1)[0]
        proxies = {
            tmp[2].lower() : tmp[2].lower() + '://' + str(tmp[0]) + ':' + str(tmp[1])
        }
        print('本次请求的代理ip:' + proxies[tmp[2].lower()])
        return proxies

In [96]:
class Spider(object):
    """
    捕获数据类
    """
    def __init__(self, url):
        self.url = url
    def show_demo(self, prms = params):
        """
        查看数据字段信息
        :prms:请求字段
        """
        return self.parse_request(self.send_request(prms))[0]
    def send_request(self, prms = {}, url = ''):
        """
        发送请求
        :prms:请求字段
        :url:请求地址
        """
        if not url:
            url = self.url
        utils = Utils()
        request = requests.get(url, headers=utils.get_headers(), proxies = utils.get_proxies_agent(), params = prms)
        return request.text
    def parse_request(self, text):
        """
        解析JSON数据
        :text:解析文本
        """
        job_list = json.loads(text)['data']['disp_data']
        return job_list
    def parse(self, text):
        """
        解析网页数据
        :text:解析文本
        """
        soup = BeautifulSoup(text,'lxml')
        job_detail = soup.find('div', class_='job-detail').p.get_text()
        return job_detail
    def add_page_ability(self, data_list):
        """
        获取需要的技能
        :data_list:数据
        """
        utils = Utils()
        for data in data_list:
            loc = utils.get_page_url(data['loc'])
            request_text = self.send_request(loc['params'],loc['url'])
            parse = self.parse(request_text)
            time.sleep(3)
            data['companydescription'] = parse
        print('页面完成...')
        return data_list

In [103]:
class Save(object):
    """
    存储类
    """
    def mysql_save(self, data):
        """
        Mysql存储数据
        :data:存储的数据
        """
        db = MySQLdb.connect("localhost", "root", "123456", "py_mysql", charset='utf8' )
        utils = Utils()
        # 使用cursor()方法获取操作游标 
        cursor = db.cursor()
        for txt in data:        
            sql = utils.get_insert_sql(txt)
            cursor.execute(sql)
            db.commit()
        db.close()
    def csv_save(self, data):
        """
        csv存储数据
        :data:存储的数据
        """
        utils = Utils()
        with open('data.csv',mode = 'a', newline='') as f:
            writer = csv.DictWriter(f, csv_headers)
            for row in data:
                writer.writerow(utils.parse_dict(row))

In [91]:
# 存储数据库
if __name__ == '__main__':
    spider = Spider(url)
    save = Save()
    utils = Utils()
    utils.get_proxies()
    for index in range(0,3):
        params['pn'] = index * 20
        request_text = spider.send_request(params)
        job_list = spider.parse_request(request_text)
        print("存储第{}页".format(index + 1))        
        save.mysql_save(job_list)
        time.sleep(3)
    print('mysql完成存储...')

[['ip', 'port', 'request_type', 'time'], ['47.98.251.15', '8118', 'HTTP', '3'], ['183.166.111.162', '9999', 'HTTP', '2'], ['119.120.5.229', '9999', 'HTTP', '2'], ['113.194.30.67', '9999', 'HTTP', '2'], ['163.204.246.190', '9999', 'HTTP', '2'], ['114.239.145.206', '9999', 'HTTP', '1'], ['110.243.4.146', '9999', 'HTTP', '0.4'], ['110.243.15.72', '9999', 'HTTP', '3'], ['183.166.21.143', '9999', 'HTTP', '1'], ['175.42.129.91', '9999', 'HTTP', '2'], ['175.42.129.149', '9999', 'HTTP', '2'], ['49.86.177.231', '9999', 'HTTP', '3'], ['175.42.158.133', '9999', 'HTTP', '3'], ['123.169.166.130', '9999', 'HTTP', '3'], ['49.70.17.226', '9999', 'HTTP', '3']]
获取代理ip成功！！！
['183.166.21.143', '9999', 'HTTP', '1']


TypeError: list indices must be integers or slices, not str

In [56]:
# 存储csv
if __name__ == '__main__':
    spider = Spider(url)
    save = Save()
    utils = Utils()
    utils.get_proxies()
    # 写入头部     
    with open('data.csv',mode = 'a', newline='') as f:
            writer = csv.DictWriter(f, csv_headers)
            writer.writeheader()
    # 写数据     
    for index in range(12,17):
        params['pn'] = index * 20
        request_text = spider.send_request(params)
        job_list = spider.parse_request(request_text)
        job_list = spider.add_page_ability(job_list)
        print("存储第{}页".format(index + 1))        
        save.csv_save(job_list)
        time.sleep(3)
    print('csv完成存储...')

页面完成...
存储第13页
页面完成...
存储第14页
页面完成...
存储第15页
页面完成...
存储第16页
页面完成...
存储第17页
csv完成存储...


In [102]:
# 测试技能字段添加函数
spider = Spider(url)
utils = Utils()
utils.get_proxies()
request_text = spider.send_request(params)
job_list = spider.parse_request(request_text)
spider.add_page_ability(job_list)

获取代理ip成功！！！
本次请求的代理ip:http://114.239.145.206:9999
本次请求的代理ip:request_type://ip:port
本次请求的代理ip:http://163.204.246.190:9999
本次请求的代理ip:http://175.42.129.149:9999
本次请求的代理ip:http://183.166.21.143:9999
本次请求的代理ip:http://183.166.21.143:9999
本次请求的代理ip:http://123.169.166.130:9999
本次请求的代理ip:http://175.42.158.133:9999
本次请求的代理ip:http://183.166.21.143:9999
本次请求的代理ip:http://123.169.166.130:9999
本次请求的代理ip:http://183.166.21.143:9999
本次请求的代理ip:http://183.166.21.143:9999
本次请求的代理ip:http://49.70.17.226:9999
本次请求的代理ip:http://175.42.129.91:9999
本次请求的代理ip:http://175.42.158.133:9999
本次请求的代理ip:http://183.166.21.143:9999
本次请求的代理ip:request_type://ip:port
本次请求的代理ip:http://183.166.111.162:9999
本次请求的代理ip:http://123.169.166.130:9999
本次请求的代理ip:http://110.243.4.146:9999
本次请求的代理ip:http://47.98.251.15:8118
页面完成...


[{'StdStg': '8055',
  'StdStl': '8',
  '_update_time': '1591386866',
  'loc': 'http://kg.baidu.com/od/4002/2004461/7a3ee10e580d5ffaeef3ff9a0460c643',
  'rloc': 'a12a82258b08f65bec0d6f7952f180b4',
  'changefreq': 'always',
  'priority': '1.0',
  'lastmod': '2020-06-06',
  'all_direct': '1',
  'resource_id': '2004461',
  'sourcelink': 'https://www.liepin.com/',
  'welfare': ['五险一金'],
  'companydescription': '工作职责: 1、负责区域内在建项目的的岩土专业设计管理工作； 2、负责各项目基坑支护、边坡挡墙等的图纸会审及优化深化； 3、从地质灾害防范等角度评估项目在开发过程中存在的问题及提出相应的解决方案； 任职资格: 1、全日制本科及以上学历，地质、岩土相关专业； 2、对云南省及周边省份的地质情况熟悉，对于边坡挡墙设计及基坑支护设计方面有较多的实践经验及一定的见解； 3、具备较强的沟通协调能力及抗压能力。',
  'dgw_site': 'www.liepin.com',
  'jobfirstclass': '岩土工程',
  'title': '岩土设计经理(雅居乐)',
  'static_pcvr': '30858',
  'is_promise': '0',
  'app_proc_time': '1591386859',
  'source': '猎聘',
  'location': '昆明',
  'second_level_label': '高级管理',
  'top_img': 'https://image0.lietou-static.com/big_/5bfe955674719dc6ed7419e903a.png',
  '@site': 'kg',
  'ori_welfare': ['五险一金'],
  'ori_welfare#num#bai

In [42]:
# 查看数据类型
spider = Spider(url)
spider.show_demo(params)

{'StdStg': '8055',
 'StdStl': '8',
 '_update_time': '1591301931',
 'loc': 'http://kg.baidu.com/od/4002/2004461/e24ac80a08b8c82e3e7f51b817dd1b98',
 'rloc': '916026bc7921472fbecfb1be5d04e3b3',
 'changefreq': 'always',
 'priority': '1.0',
 'lastmod': '2020-06-05',
 'all_direct': '1',
 'resource_id': '2004461',
 'sourcelink': 'https://www.liepin.com/',
 'companydescription': '苏州英派斯健身俱乐部有限公司石路天虹馆办公室地址位于中国园林之城，人间天堂苏州，苏州 苏州市广济南路219号（星昶城购物中心内10层），于2017年03月20日在苏州市姑苏区市场监督管理局苏州国家历史文化名城保护区市场监督管理局注册成立，注册资本为0万元人民币，在公司发展壮大的2年里，我们始终为客户提供好的产品和技术支持、健全的售后服务，我公司主要经营承接总公司业务。（依法须经批准的项目，经相关部门批准后方可开展经营活动），我们有好的产品和专业的销售和技术团队，我公司属于苏州体育黄页行业，如果您对我公司的产品服务有兴趣，期待您在线留言或者来电咨询',
 'dgw_site': 'www.liepin.com',
 'jobfirstclass': '咨询/法律/翻译/商标/专利',
 'title': '销售代表(苏州英派斯健身俱乐部有限公司石路天虹馆)',
 'static_pcvr': '31502',
 'is_promise': '0',
 'app_proc_time': '1591300486',
 'source': '猎聘',
 'location': '苏州',
 'second_level_label': '销售',
 'top_img': 'https://image0.lietou-static.com/big_/5db54fa6cd52473b94afb94f08u.jpg',
 '@site': 'kg

In [58]:
# 测试路径
utils = Utils()
utils.get_path()

'F:\\Python'